In [1]:
pip install tabula-py

     |████████████████████████████████| 12.0 MB 18.5 MB/s eta 0:00:01
You should consider upgrading via the '/Library/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tabula

In [6]:
tables = tabula.read_pdf("September_2021.pdf", pages=4, multiple_tables=True)

In [7]:
tables[0]

,Unnamed: 0,Unnamed: 1,Sales by Product Type,Unnamed: 2
0,NaN,NaN,September 1 - September 30,NaN
1,NaN,Product Type,Pounds Sold Fluid Ounces Sold,Total Sales
2,Flower,NaN,"5,889.67","$18,050,603.36"
3,Shake/Trim,NaN,677.99,"$1,726,974.37"
4,Concentrate,NaN,301.62,"$3,090,407.89"
5,Inhalable Compound Concentrate,NaN,1.41,"$17,193.40"
6,Vape Cartridge,NaN,780.45,"$9,254,607.57"
7,Kief,NaN,1.01,"$7,240.32"
8,Infused-Edible,NaN,"35,669.53","$3,950,181.04"
9,Infused Non-Edible Solid,NaN,424.9,"$163,485.29"


In [8]:
tables[1]

,Additional Sales Information
0,September 1 - September 30
1,Category Amount
2,"Sales To Date (as of September 30) $1,171,171,..."
3,"Sales Deliveries $1,450,837.24"
4,Average Retail Flower Price (oz.) $191.55


In [9]:
tables[2]

,Unnamed: 0
0,Plants
1,Active Plants
2,"As of September 30, 2021"
3,Plant State Number
4,NaN
5,"Immature 59,846"
6,"Vegetative 107,492"
7,"Flowering 267,018"


In [10]:
tables[3]

,Destroyed & Harvested Plants
0,September 1 – September 30
1,Plant State Number
2,NaN
3,"Immature - Destroyed 29,640"
4,"Vegetative - Destroyed 18,469"
5,"Flowering - Destroyed 7,660"
6,NaN
7,"Plants Harvested 75,278"
